# Solución etapa 3 - FE - V2

In [31]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [32]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')


## Feature eng

### Target

In [33]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [34]:
tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1
(9951, 7)


In [35]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1      1
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0   2020      1      1
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0   2020      1      1
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1   2020      1      1
(9951, 10)


In [36]:
# Tiempo medio entre fallos por fecha

# Filtrar solo los registros con fallos
fallos = historicos_ordenes_cleaned[historicos_ordenes_cleaned['Fallo'] == 1].copy()

# Ordenar por fecha
fallos = fallos.sort_values(by=['ID_Equipo', 'Fecha'])

# Calcular los intervalos entre fallos consecutivos
fallos['Intervalo'] = fallos.groupby('ID_Equipo')['Fecha'].diff()
fallos['Intervalo_Horas'] = fallos['Intervalo'].dt.total_seconds() / 3600

# Calcular el Tiempo medio entre fallos por equipo
mtbf_por_idequipo = fallos.groupby('ID_Equipo')['Intervalo_Horas'].mean().reset_index()
mtbf_por_idequipo.rename(columns={'Intervalo_Horas': 'Tiempo_medio_entre_fallos(h)'}, inplace=True)

tabl(mtbf_por_idequipo)

      ID_Equipo    Tiempo_medio_entre_fallos(h)
--  -----------  ------------------------------
 0            1                        1164
 1            2                         667.462
 2            3                         984.167
 3            4                        1300.43
 4            5                        1341.71
(499, 2)


In [37]:
# Mezclar historicos_ordenes_cleaned + mtbf_por_idequipo

historicos_mtbf = pd.merge(historicos_ordenes_cleaned, mtbf_por_idequipo, on=['ID_Equipo'], how='left')
tabl(historicos_mtbf)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día    Tiempo_medio_entre_fallos(h)
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----  ------------------------------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1      1                        1220.12
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0   2020      1      1                         772.5
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0   2020      1      1                         752
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1                         699.077
 4         

In [38]:
# Tiempo medio entre fallos por año - mes

# Filtrar solo los registros con fallos
fallos = historicos_ordenes_cleaned[historicos_ordenes_cleaned['Fallo'] == 1].copy()

# Ordenar por fecha
fallos = fallos.sort_values(by=['ID_Equipo', 'Fecha'])

fallos['Diferencia_Horas'] = fallos.groupby('ID_Equipo')['Fecha'].diff().dt.total_seconds() / 3600

promedio_horas_entre_fallos_por_mes = (
    fallos.groupby(['ID_Equipo', 'Año', 'Mes','Día'])['Diferencia_Horas']
          .mean()
          .reset_index(name='Promedio_Horas_Entre_Fallos_por_mes')
)

tabl(promedio_horas_entre_fallos_por_mes)

      ID_Equipo    Año    Mes    Día    Promedio_Horas_Entre_Fallos_por_mes
--  -----------  -----  -----  -----  -------------------------------------
 0            1   2020      2      4                                    nan
 1            1   2020      2     25                                    507
 2            1   2020      3     17                                    492
 3            1   2020      6     12                                   2100
 4            1   2020      7     26                                   1034
(4895, 5)


In [39]:
# Mezclar historicos_ordenes_cleaned + promedio_horas_entre_fallos_por_mes

historicos_promedio_horas_entre_fallos_por_mes = pd.merge(historicos_ordenes_cleaned, promedio_horas_entre_fallos_por_mes, on=['ID_Equipo','Año','Mes'], how='inner')
tabl(historicos_promedio_horas_entre_fallos_por_mes)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día_x    Día_y    Promedio_Horas_Entre_Fallos_por_mes
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -------  -------  -------------------------------------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1        1        4                                    nan
 1           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1        1        1                                    nan
 2           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1   2020      1        1        1                                    nan
 3           6          477  2020-01-01 05:00:00                1519.65         

In [40]:
# Verificamos nulos
print(historicos_promedio_horas_entre_fallos_por_mes.isna().sum())
historicos_promedio_horas_entre_fallos_por_mes.dropna(inplace=True)

ID_Orden                                  0
ID_Equipo                                 0
Fecha                                     0
Costo_Mantenimiento                       0
Duracion_Horas                            0
Ubicacion                                 0
Fallo                                     0
Año                                       0
Mes                                       0
Día_x                                     0
Día_y                                     0
Promedio_Horas_Entre_Fallos_por_mes    1074
dtype: int64


In [41]:
# Eliminar columnas sin valor de historicos_mtbf
historicos_mtbf_clean = historicos_mtbf.drop(columns=['ID_Orden','Costo_Mantenimiento','Duracion_Horas','Fecha'])
tabl(historicos_mtbf_clean)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día    Tiempo_medio_entre_fallos(h)
--  -----------  ------------  -------  -----  -----  -----  ------------------------------
 0           36  Planta Norte        0   2020      1      1                        1220.12
 1          244  Planta Norte        0   2020      1      1                         772.5
 2          297  Planta Norte        0   2020      1      1                         752
 3          431  Planta Sur          1   2020      1      1                         699.077
 4          403  Planta Norte        1   2020      1      1                        1368.86
(9951, 7)


In [42]:
# Eliminar columnas sin valor de historicos_promedio_horas_entre_fallos_por_mes
historicos_hefbm_clean = historicos_promedio_horas_entre_fallos_por_mes.drop(columns=['ID_Orden','Costo_Mantenimiento','Duracion_Horas','Fecha'])
tabl(historicos_hefbm_clean)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día_x    Día_y    Promedio_Horas_Entre_Fallos_por_mes
--  -----------  ------------  -------  -----  -----  -------  -------  -------------------------------------
 7          286  Planta Este         1   2020      1        1        4                                     83
 8          286  Planta Este         1   2020      1        1       30                                    609
10          286  Planta Oeste        0   2020      1        1        4                                     83
11          286  Planta Oeste        0   2020      1        1       30                                    609
15          278  Planta Norte        1   2020      1        1       22                                    491
(10727, 8)


In [43]:
historicos_hefbm_clean.groupby(['ID_Equipo','Año','Mes']).count().loc[1]

Ubicacion  Fallo  Día_x  Día_y  Promedio_Horas_Entre_Fallos_por_mes
Año  Mes                                                                     
2020 2            3      3      3      3                                    3
     3            3      3      3      3                                    3
     6            1      1      1      1                                    1
     7            1      1      1      1                                    1
     8            3      3      3      3                                    3
     12           1      1      1      1                                    1
2021 1            1      1      1      1                                    1

> Coincidencia a nivel de mes entre historico y registro

### registros_condiciones

In [44]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Año    Mes    Día
--  -----------  -------------------  ---------------  ----------------  ------------------  -----  -----  -----
 0          260  2020-01-01 00:00:00           101.43              1.22               71849   2020      1      1
 2          443  2020-01-01 02:00:00            76.04              2.93               33106   2020      1      1
 3          281  2020-01-01 03:00:00            97.09              3.25               31744   2020      1      1
 4          427  2020-01-01 04:00:00           139.42              4.22               78104   2020      1      1
 5           48  2020-01-01 05:00:00            98.77              2.73               89664   2020      1      1
(8999, 8)


In [45]:
registros_condiciones_cleaned.groupby(['ID_Equipo','Año','Mes']).count().loc[1]

Fecha  Temperatura_C  Vibracion_mm_s  Horas_Operativas  Día
Año  Mes                                                             
2020 1        2              2               2                 2    2
     3        2              2               2                 2    2
     4        3              3               3                 3    3
     6        1              1               1                 1    1
     7        2              2               2                 2    2
     8        2              2               2                 2    2
     10       1              1               1                 1    1
     12       2              2               2                 2    2

In [46]:
# Agrupar por 'ID_Equipo', 'Año' y 'Mes', y calcular los promedios
promedios_por_equipo_mes = (
    registros_condiciones_cleaned.groupby(['ID_Equipo', 'Año', 'Mes','Día'])[['Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]
      .mean()
      .reset_index()
)

# Renombrar las columnas para claridad
promedios_por_equipo_mes.rename(columns={
    'Temperatura_C': 'Promedio_Temperatura_C',
    'Vibracion_mm_s': 'Promedio_Vibracion_mm_s',
    'Horas_Operativas': 'Promedio_Horas_Operativas'
}, inplace=True)

tabl(promedios_por_equipo_mes)

      ID_Equipo    Año    Mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas
--  -----------  -----  -----  -----  ------------------------  -------------------------  ---------------------------
 0            1   2020      1      2                     93.8                        0.7                         80054
 1            1   2020      1     21                    102.99                       2.31                        74797
 2            1   2020      3      2                    145.97                       1.57                        95240
 3            1   2020      3     16                    147.18                       2.49                         6872
 4            1   2020      4     17                     58.93                       3.54                        28186
(8793, 7)


In [47]:
promedios_por_equipo_mes.isna().sum()
promedios_por_equipo_mes.dropna(inplace=True)
promedios_por_equipo_mes.shape

(8755, 7)

### Unir merged_hitorico_registro + promedios + caracteristicas

In [48]:
# Unir datasets
final_data = pd.merge(historicos_hefbm_clean, promedios_por_equipo_mes, on=['ID_Equipo','Año','Mes'], how='inner')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

tabl(final_data)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día_x    Día_y    Promedio_Horas_Entre_Fallos_por_mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  ------------  -------  -----  -----  -------  -------  -------------------------------------  -----  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0          286  Planta Este         1   2020      1        1        4                                     83     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 1          286  Planta Este         1   2020      1        1       30                                    609     29                     57.58      

In [49]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Equipo','Año', 'Mes', 'Día_x', 'Día_y'], inplace=True)

tabl(final_data)

    Ubicacion       Fallo    Promedio_Horas_Entre_Fallos_por_mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  -------------------------------------  -----  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Este         1                                     83     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 1  Planta Este         1                                    609     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 2  Planta Oeste        0                   

### Duplicados

In [50]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)


Duplicados final_data:
 1659


In [51]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())


Nulos final_data:
 Ubicacion                              0
Fallo                                  0
Promedio_Horas_Entre_Fallos_por_mes    0
Día                                    0
Promedio_Temperatura_C                 0
Promedio_Vibracion_mm_s                0
Promedio_Horas_Operativas              0
Tipo_Equipo                            0
Fabricante                             0
Modelo                                 0
Potencia_kW                            0
Horas_Recomendadas_Revision            0
dtype: int64


In [52]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

[]

In [53]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

if columnas_con_nulos and len(columnas_con_nulos)>0:
    final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [54]:
tabl(final_data)

    Ubicacion       Fallo    Promedio_Horas_Entre_Fallos_por_mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  -------------------------------------  -----  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Este         1                                     83     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 1  Planta Este         1                                    609     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 2  Planta Oeste        0                   

In [55]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13336 entries, 0 to 13335
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Ubicacion                            13336 non-null  object 
 1   Fallo                                13336 non-null  int64  
 2   Promedio_Horas_Entre_Fallos_por_mes  13336 non-null  float64
 3   Día                                  13336 non-null  int32  
 4   Promedio_Temperatura_C               13336 non-null  float64
 5   Promedio_Vibracion_mm_s              13336 non-null  float64
 6   Promedio_Horas_Operativas            13336 non-null  float64
 7   Tipo_Equipo                          13336 non-null  object 
 8   Fabricante                           13336 non-null  object 
 9   Modelo                               13336 non-null  object 
 10  Potencia_kW                          13336 non-null  float64
 11  Horas_Recomendadas_Revision 

### Volvemos a explorar con columnas codificadas


In [56]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)


Numerical columns
 ['Fallo', 'Promedio_Horas_Entre_Fallos_por_mes', 'Día', 'Promedio_Temperatura_C', 'Promedio_Vibracion_mm_s', 'Promedio_Horas_Operativas', 'Potencia_kW', 'Horas_Recomendadas_Revision']

Categorical columns
 ['Ubicacion', 'Tipo_Equipo', 'Fabricante', 'Modelo']


In [57]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded =final_data.copy()
final_data_coded[categorical_columns] =final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)   
tabl(final_data_coded)

      Ubicacion    Fallo    Promedio_Horas_Entre_Fallos_por_mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas    Tipo_Equipo    Fabricante    Modelo    Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------  -------------------------------------  -----  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0            0        1                                     83     29                     57.58                       3.65                          250              1             1         2           3432                           8213
 1            0        1                                    609     29                     57.58                       3.65                          250              1             1         2           3432                           8213
 2            2        0                        

In [58]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)


Feature scores



,Feature,Score
1,Promedio_Horas_Entre_Fallos_por_mes,89.37
7,Fabricante,22.39
10,Horas_Recomendadas_Revision,4.47
0,Ubicacion,4.26
3,Promedio_Temperatura_C,2.13
6,Tipo_Equipo,1.22
9,Potencia_kW,1.04
8,Modelo,0.83
4,Promedio_Vibracion_mm_s,0.83
2,Día,0.61


In [59]:
# Eliminamos columnas sin peso

columns_to_drop = feature_scores[feature_scores['Score'] == 0]['Feature']
final_data.drop(columns=columns_to_drop, inplace=True)
tabl(final_data)

    Ubicacion       Fallo    Promedio_Horas_Entre_Fallos_por_mes    Día    Promedio_Temperatura_C    Promedio_Vibracion_mm_s    Promedio_Horas_Operativas  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  ------------  -------  -------------------------------------  -----  ------------------------  -------------------------  ---------------------------  -------------  ------------  --------  -------------  -----------------------------
 0  Planta Este         1                                     83     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 1  Planta Este         1                                    609     29                     57.58                       3.65                          250  Generador      GE            Y200               3432                           8213
 2  Planta Oeste        0                   

## Exportar 

In [60]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v2.csv', index=False)